In [ ]:
# first convert ods to csv

import pandas as pd
import pickle
from aksharamukha import transliterate
import re
import os

read index

In [ ]:
index_df = pd.read_excel('/home/bhikkhu/Bodhirasa/Dropbox/dpd/inflection list generator/declensions & conjugations.xlsx', sheet_name="index", dtype=str)
index_df.fillna("", inplace=True)
# index_df["inflection name"] = index_df["inflection name"].str.replace(" ", "_") #only needed if using as variable
index_df_length = len(index_df)

index_df.columns
index_dict = dict(zip(index_df.iloc[:, 0], index_df.iloc[:, 2]))

generate df's of each inflection pattern

In [ ]:
print("~" * 40)
print("inflection generator.ipynb")
print("~" * 40)

inflection_df = pd.read_excel('/home/bhikkhu/Bodhirasa/Dropbox/dpd/inflection list generator/declensions & conjugations.xlsx', sheet_name="declensions", dtype=str)

inflection_df = inflection_df.shift(periods=2)

inflection_df.columns = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "AA", "AB", "AC", "AD", "AE", "AF", "AG", "AH", "AI", "AJ", "AK", "AL", "AM", "AN", "AO", "AP", "AQ", "AR", "AS", "AT", "AU", "AV", "AW", "AX", "AY", "AZ", "BA", "BB", "BC", "BD", "BE", "BF", "BG", "BH", "BI", "BJ", "BK", "BL", "BM", "BN", "BO", "BP", "BQ", "BR", "BS", "BT", "BU", "BV", "BW", "BX", "BY", "BZ", "CA", "CB", "CC", "CD", "CE", "CF", "CG", "CH", "CI", "CJ", "CK", "CL", "CM", "CN", "CO", "CP", "CQ", "CR", "CS", "CT", "CU", "CV", "CW", "CX", "CY", "CZ", "DA", "DB", "DC", "DD", "DE", "DF", "DG", "DH", "DI", "DJ", "DK"]
inflection_df.fillna("", inplace=True)

pattern_changed = []

for row in range(index_df_length): #index_df_length
	inflection_name = index_df.iloc[row,0]
	cell_range = index_df.iloc[row,1]
	like = index_df.iloc[row,2]
	irreg = index_df.iloc[row,3]

	col_range_1 = re.sub("(.+?)\d*\:.+", "\\1", cell_range)
	col_range_2 = re.sub(".+\:(.[A-Z]*)\d*", "\\1", cell_range)
	row_range_1 = int(re.sub(".+?(\d{1,3}):.+", "\\1", cell_range))
	row_range_2 = int(re.sub(".+:.+?(\d{1,3})", "\\1", cell_range))

	inflection_df_filtered = inflection_df.loc[row_range_1:row_range_2, col_range_1:col_range_2]
	inflection_df_filtered.Name =  f"{inflection_name}"

	inflection_df_filtered.reset_index(drop=True, inplace=True)

	inflection_df_filtered.iloc[0,0] = ""

	# replace header

	new_header = inflection_df_filtered.iloc[0] #grab the first row for the header
	inflection_df_filtered = inflection_df_filtered[1:] #take the data less the header row
	inflection_df_filtered.columns = new_header #set the header row as the df header

	# replace index

	inflection_df_filtered.index = inflection_df_filtered.iloc[0:,0]
	inflection_df_filtered = inflection_df_filtered.iloc[:, 1:]

	# remove unmaed column headers

	inflection_df_filtered = inflection_df_filtered.rename(columns=lambda x: re.sub('Unnamed.*','',x))

	# test

	try:
		old = pd.read_csv(f"output/patterns/{inflection_name}.csv", sep="\t", index_col=0, na_filter=False) #, header=0
		old.fillna("", inplace=True)
		old = old.rename(columns=lambda x: re.sub('Unnamed.*','',x))
	except:
		print(f"{inflection_name} - doesn't exist - added")
		pattern_changed.append(inflection_name)
		inflection_df_filtered.to_csv(f"output/patterns/{inflection_name}.csv", sep="\t")

	if inflection_df_filtered.equals(old):
		continue
	elif inflection_name in pattern_changed:
		continue
	elif not inflection_df_filtered.equals(old):
		print(f"{inflection_name} - different - updated")
		inflection_df_filtered.to_csv(f"output/patterns/{inflection_name}.csv", sep="\t")
		pattern_changed.append(inflection_name)

if pattern_changed == []:
	print("all patterns identical")
if pattern_changed != []:
	print("~" * 40)
	print(f"the following patterns have changes and will be generated\n{pattern_changed}")


create dpd_df

In [ ]:
dpd_df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/dpd-full.csv", sep="\t", dtype=str)
dpd_df.fillna("", inplace=True) 
dpd_df_length = len(dpd_df)

test for missing stem & pattern

In [ ]:
#empty stem

error = False

for row in range(dpd_df_length):
	headword = dpd_df.loc[row, "Pāli1"]
	stem = dpd_df.loc[row, "Stem"]
	pattern = dpd_df.loc[row, "Pattern"]
	
	if stem == "":
		print(f"stem error: {headword} has no stem!")
		error = True
	if stem != "-" and pattern == "":
		print(f"pattern error: {headword} has no pattern!")
		error = True

if error == True:
	print("~" * 40)
	input("there are stem & pattern errors, please fix them before continuiing")
else:
	print("~" * 40)
	print("no stem & pattern errors found")

test for wrong patterns

In [ ]:
print("~" * 40)
print("testing for wrong patterns")

index_patterns = index_df["inflection name"].values.tolist()

error = False

fixme = ""

for row in range(dpd_df_length):
	headword =  dpd_df.loc[row, "Pāli1"]
	stem = dpd_df.loc[row, "Stem"]
	pattern = dpd_df.loc[row, "Pattern"]

	if stem == "-":
		pass
		# print(f"pass. {headword} stem ")
	elif stem == "!":
		pass
		# print(f"pass {headword} stem !")
	elif pattern in index_patterns:
		pass
		# print(f"pass {headword} {pattern} exists")
	elif pattern not in index_patterns:
		print("~" * 40)
		print(f"oops! {headword}. {pattern}")
		error = True
	else:
		pass

if error == True:
	print("~" * 40)
	input("wrong patterns - fix 'em!")

test for differences in stem and pattern

In [ ]:
changed = []

print("~" * 40)
print("testing for changes in stem and pattern")

for row in range(dpd_df_length): #dpd_df_length
	headword = dpd_df.loc[row, "Pāli1"]
	stem = dpd_df.loc[row, "Stem"]
	pattern = dpd_df.loc[row, "Pattern"]
	new = f"{headword} {stem} {pattern}"

	try:
		pickle_file = open(f"output/pickle test/{headword}","rb")
		old = pickle.load(pickle_file)
		pickle_file.close()
	except:
		print(f"{headword} - doesn't exist - added")
		changed.append(headword)
		pickle_file = open(f"output/pickle test/{headword}","wb")
		pickle.dump(new,pickle_file)
		pickle_file.close()
		continue

	if old == new:
		continue
	elif old in changed:
		continue
	elif old != new:
		print(f"{headword} {stem} {pattern} - not identical")
		changed.append(headword)
		pickle_file = open(f"output/pickle test/{headword}","wb")
		pickle.dump(new,pickle_file)
		pickle_file.close()

print("~"*40)
if changed == []:
	print("no headwords stems or patterns changed")
if changed != []:
	print(f"the following patterns have changes and will be generated:\n{changed}")


test if inflections exist

In [ ]:
synonyms_not_exist = []

for row in range(dpd_df_length): #dpd_df_length
	headword = dpd_df.loc[row, "Pāli1"]
	
	try:
		with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/inflection list generator/output/synonyms/{headword}", "rb") as syn_file:
			pass
	
	except:
		print(f"synoym file for - {headword} - doesn't exist")
		synonyms_not_exist.append(headword)

print("~"*40)

if synonyms_not_exist == []:
	print("no missing synonym files")
if synonyms_not_exist != []:
	print(f"the following synonym files are missing and will be generated:\n{synonyms_not_exist}")

generate all html inflection tables

In [ ]:
print("~" * 40)
print("generating inflection tables")
print("~" * 40)

indeclinables = ["abbrev", "abs", "ger", "ind", "inf", "prefix"]
conjugations = ["aor", "cond", "fut", "imp", "imperf", "opt", "perf", "pr"]
declensions = ["adj", "card", "cs", "fem", "letter", "masc", "nt", "ordin", "pp", "pron", "prp", "ptp", "root", "suffix", "ve"]

row = 0

for row in range(dpd_df_length): #dpd_df_length
	headword = dpd_df.loc[row, "Pāli1"]
	headword_clean = re.sub(" \d*$", "", headword)
	stem = dpd_df.loc[row, "Stem"]
	if stem == "*":
		stem = ""
	pattern = dpd_df.loc[row, "Pattern"]
	pos = dpd_df.loc[row, "POS"]
	metadata = dpd_df.loc[row, "Metadata"]
	meaning = dpd_df.loc[row, "Meaning IN CONTEXT"]

	if headword in changed or pattern in pattern_changed or headword in synonyms_not_exist:
		print(f"{row}/{dpd_df_length}\t{headword}")

		with open(f"output/html/{headword}.html", "w") as html_table:
				
			if stem == "-":
				html_table.write(f"<p><b>{headword_clean}</b> is indeclinable")

			elif stem == "!":
				html_table.write(f"<p>click on <b>{pattern}</b> for inflection table")

			else:
				df = pd.read_csv(f"output/patterns/{pattern}.csv", sep="\t", index_col=0)
				df.fillna("", inplace=True, axis=0)
				
				df_rows = df.shape[0]
				df_columns = df.shape[1]

				for rows in range(0, df_rows): 
					for columns in range(0, df_columns, 2): #1 to 0
						
						html_cell = df.iloc[rows, columns]
						syn_cell = df.iloc[rows, columns]	

						if html_cell == "sg" or html_cell == "pl":
							pass

						else:
							html_cell = re.sub(r"(.+)", f"<b>\\1</b>", html_cell) # add bold
							html_cell = re.sub(r"(.+)", f"{stem}\\1", html_cell) # add stem
							html_cell = re.sub(r"\n", "<br>", html_cell) # add line breaks
							df.iloc[rows, columns] = html_cell
							
							syn_cell = re.sub(r"(.+)", f"{stem}\\1", syn_cell)
							search_string = re.compile("\n", re.M)
							replace_string = " "
							matches = re.sub(search_string, replace_string, syn_cell)
							# synonyms += matches + " "
				
				column_list = []
				for i in range(1, df_columns, 2):
					column_list.append(i)

				df.drop(df.columns[column_list], axis=1, inplace=True)
				table = df.to_html(escape=False)
				table = re.sub("Unnamed.+", "", table)
				table = re.sub("NaN", "", table)

				# write header info

				if index_dict[pattern] != "":
					if pos in declensions:
						heading = (f"""<p><b>{headword_clean}</b> is <b>{pattern}</b> declension like <b>{index_dict[pattern]}</b></p>""")
					if pos in conjugations:
						heading = (f"""<p><b>{headword_clean}</b> is <b>{pattern}</b> conjugation like <b>{index_dict[pattern]}</b></p>""")

				if index_dict[pattern] == "":
					if pos in declensions:
						heading = (f"""<p><b>{headword_clean}</b> is <b>{pattern}</b> irregular declension</p>""")
					if pos in conjugations:
						heading = (f"""<p><b>{headword_clean}</b> is <b>{pattern}</b> irregular conjugation</p>""")
				
				html = heading + table 
				html_table.write(html)



generate all synonyms

In [ ]:
print("~" * 40)
print("generating synonyns")
print("~" * 40)

new_synonyms_dict = {}

for row in range(dpd_df_length): #dpd_df_length
	headword = dpd_df.loc[row, "Pāli1"]
	headword_clean = re.sub(" \d*$", "", headword)
	stem = dpd_df.loc[row, "Stem"]
	if stem == "*":
		stem = ""
	pattern = dpd_df.loc[row, "Pattern"]
	pos = dpd_df.loc[row, "POS"]
	metadata = dpd_df.loc[row, "Metadata"]
	meaning = dpd_df.loc[row, "Meaning IN CONTEXT"]
	variant = dpd_df.loc[row, "Variant – same constr or diff reading"]

	synonyms = ""

	if headword in changed or pattern in pattern_changed or headword in synonyms_not_exist:
	# if 1== 1:
		print(f"{row} {headword}")

		if stem == "-":
			synonyms += headword_clean + " "

		elif stem == "!":
			synonyms += headword_clean + " "

		else:
			synonyms += headword_clean + " "

			variant = re.sub(",", " ", variant)
			synonyms += variant + " "
		
			try:
				df = pd.read_csv(f"output/patterns/{pattern}.csv", sep="\t", header=None)
				df.fillna("", inplace=True)
				df_rows = df.shape[0]
				df_columns = df.shape[1]

				for rows in range(1, df_rows):
					for columns in range(1, df_columns, 2):
						line = df.iloc[rows, columns]
						line = re.sub(r"(.+)", f"{stem}\\1", line)
						search_string = re.compile("\n", re.M)
						replace_string = " "
						matches = re.sub(search_string, replace_string, line)

						synonyms += matches + " "

			except:
				with open("/home/bhikkhu/Bodhirasa/Dropbox/dpd/errorlogs/inflection generator errorlog.txt", "a") as error_log:
					error_log.write(f"error on: {headword}\n")
		
		this_word_synonyms = {headword : synonyms}
		new_synonyms_dict.update(this_word_synonyms)

if new_synonyms_dict != {}:
	new_synonyms_df = pd.DataFrame.from_dict(new_synonyms_dict, orient='index')
	new_synonyms_df.to_csv("output/new synonyms.csv", sep="\t", header=False)

else:
	print("no new synonyms")

transcribe new synonyms

In [ ]:
if new_synonyms_dict != {}:
  print("~" * 40)

  new_synonyms = open("output/new synonyms.csv", "r")
  new_synonyms_read = new_synonyms.read()
  new_synonyms.close()

  new_synonyms_translit = open("output/new synonyms translt.csv", "w")

  print("converting synonyms to sinhala")
  sinhala = transliterate.process("IAST","Sinhala", new_synonyms_read, post_options =['SinhalaPali', 'SinhalaConjuncts'])
  
  print("converting synonyms to devanagari")
  devanagari = transliterate.process("IAST","Devanagari",new_synonyms_read, post_options = ['DevanagariAnusvara'])

  roman = new_synonyms_read.split("\n")[:-1]
  sinhala = sinhala.split("\n")
  devanagari = devanagari.split("\n")

  for i in zip(roman, sinhala, devanagari):	
    new_synonyms_translit.write(i[0]+i[1].split("\t")[1]+i[2].split("\t")[1]+"\n")

  new_synonyms_translit.close()

else:
	print("no new synonyms to transcribe")

combine dataframes

In [ ]:
diff = pd.DataFrame()

if new_synonyms_dict != {}:
    all_synonyms_translit = pd.read_csv("output/all synonyms translt.csv", header=None, sep="\t")

    new_synonyms_translit = pd.read_csv("output/new synonyms translt.csv", header=None, sep="\t")

    diff = pd.merge(all_synonyms_translit, new_synonyms_translit, on=[0], how='outer', indicator='exists')
    diff.to_csv("output/diff.csv", sep="\t", index=None)

    # copy changes

    test1 = diff["exists"] == "both"
    test2 = diff["1_y"] != ""
    filter = test1 & test2
    diff.loc[filter, "1_x"] = diff.loc[filter, "1_y"]

    # add new

    test1 = diff["exists"] == "right_only"
    test2 = diff["1_y"] != ""
    filter = test1 & test2
    diff.loc[filter, "1_x"] = diff.loc[filter, "1_y"]

    # !!! how to delete non existent

    # drop columns and write to csv

    diff.drop(columns=["1_y", "exists"], inplace=True)

    diff.to_csv("output/all synonyms translt.csv", sep="\t", index=None)

else:
    print("all synonyms translt.csv unchanged")

export to pickle

In [ ]:
print("~" * 40)
print("exporting synonyms to pickle")


all_synonyms = diff

length = len(all_synonyms)

for row in range(length):

	headword = all_synonyms.iloc[row, 0]
	synonyms = all_synonyms.iloc[row, 1]

	# !!! how to delete headword when no longer exists	??? 
	
	if headword in new_synonyms_dict.keys():
		print(headword)

		synonyms_list = synonyms.split()

		# add ṁ version

		for word in synonyms_list:
			if 'ṃ' in word:
				wordṁ = re.sub("ṃ", "ṁ", word) 
				synonyms_list.append(wordṁ)

		synonyms_list = list(dict.fromkeys(synonyms_list))

		with open(f"output/synonyms/{headword}", "wb") as text_file:
			pickle.dump(synonyms_list, text_file)

all changes since last run

In [ ]:
# # pickle_file = open("output/all changes since last run", "rb")
# # old_changes = pickle.load(pickle_file)
# # pickle_file.close
# # print(old_changes)

# # all_changes = old_changes
# all_changes = []
# all_changes += new_synonyms_dict.keys()
# all_changes += changed

# pickle_file = open("output/all changes since last run", "ab+")
# pickle.dump(all_changes, pickle_file)
# pickle_file.close
